<a href="https://colab.research.google.com/github/kimdonggyu2008/deep_daiv_-/blob/main/auto_vc_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install wavenet_vocoder

  Preparing metadata (setup.py) ... done
  Created wheel for wavenet_vocoder: filename=wavenet_vocoder-0.1.1-py3-none-any.whl size=12642 sha256=6255ce3ee820c0e1a36525e8ee47f38025d92ff60811bd623084c64a8640e3e0
  Stored in directory: /root/.cache/pip/wheels/4f/a4/7b/f1d21f96be36a13e9c3948e8c28792bf8962da19781abd9dc8
Successfully built wavenet_vocoder


In [ ]:
import os
import pickle
import numpy as np
import soundfile as sf
from scipy import signal
from scipy.signal import get_window
from librosa.filters import mel
from numpy.random import RandomState
import torch
import torch.nn as nn
from collections import OrderedDict
import torch.nn.functional as F
from torch.utils import data
from multiprocessing import Process, Manager
import argparse
from torch.backends import cudnn
import time
import datetime

In [ ]:
#make_spect.py
def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq #차단할 주파수 대역 지정
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a #버터워스 고역 필터 계수


def pySTFT(x, fft_length=1024, hop_length=256): #stft실행

    x = np.pad(x, int(fft_length//2), mode='reflect') #패딩, 불연속성 완화

    noverlap = fft_length - hop_length #중첩된 샘플 수 계산
    shape = x.shape[:-1]+((x.shape[-1]-noverlap)//hop_length, fft_length)#??
    strides = x.strides[:-1]+(hop_length*x.strides[-1], x.strides[-1])#스트라이드 갯수 계산
    result = np.lib.stride_tricks.as_strided(x, shape=shape,
                                             strides=strides)#?? 윈도우변환??

    fft_window = get_window('hann', fft_length, fftbins=True) #해닝 창 함수 적용
    result = np.fft.rfft(fft_window * result, n=fft_length).T #ftf후 rftf적용

    return np.abs(result)


In [ ]:
mel_basis = mel(sr=16000,n_fft=1024,  n_mels=80, fmin=90, fmax=7600).T #멜 스펙트로그램 생성
#신호의 저주파 성분 제거
min_level = np.exp(-100 / 20 * np.log(10))
#작은 신호를 0말고 최솟값으로 변환
b, a = butter_highpass(30, 16000, order=5)
#고역필터 계수 지정

# audio file directory
rootDir = '/content/drive/MyDrive/코딩공부/deep_daiv/dataset/wavs'
# spectrogram directory
targetDir = '/content/drive/MyDrive/코딩공부/deep_daiv/dataset/spmel'

# audio file directory
# rootDir = '/content/drive/MyDrive/코딩공부/deep_daiv/dataset/wavs'
# spectrogram directory
# targetDir = '/content/drive/MyDrive/코딩공부/deep_daiv/results'

#  audio file directory
# rootDir = './wavs'
# # spectrogram directory
# targetDir = './spmel'



dirName, subdirList, _ = next(os.walk(rootDir))
print('Found directory: %s' % dirName)


for subdir in sorted(subdirList): #데이터 파일 확인
    print(subdir)
    if not os.path.exists(os.path.join(targetDir, subdir)):#타겟 디렉토리 없으면 만듦
        os.makedirs(os.path.join(targetDir, subdir))
    _,_, fileList = next(os.walk(os.path.join(dirName,subdir)))#순차적 방문
    prng = RandomState(int(subdir[1:]))
    for fileName in sorted(fileList):
        # Read audio file
        x, fs = sf.read(os.path.join(dirName,subdir,fileName))#오디오파일 읽어오기
        # Remove drifting noise
        y = signal.filtfilt(b, a, x)#만든 고역필터 적용
        # Ddd a little random noise for model roubstness
        wav = y * 0.96 + (prng.rand(y.shape[0])-0.5)*1e-06
        # Compute spect
        D = pySTFT(wav).T #stft실행 후 스펙트로그램 생성
        # Convert to mel and normalize
        D_mel = np.dot(D, mel_basis)#스펙트로그램과 필터 뱅크 곱함 ??
        D_db = 20 * np.log10(np.maximum(min_level, D_mel)) - 16 #최소값 지정
        S = np.clip((D_db + 100) / 100, 0, 1)#결과값을 0~1로 제한해서 클리핑 제한
        # save spect
        np.save(os.path.join(targetDir, subdir, fileName[:-4]),
                S.astype(np.float32), allow_pickle=False)#멜스펙트로그램 저장

Found directory: /content/drive/MyDrive/코딩공부/deep_daiv/dataset/wavs
p225
p226
p227
p228


In [ ]:
#model_bl.py

class D_VECTOR(nn.Module):
    """d vector speaker embedding."""
    def __init__(self, num_layers=3, dim_input=40, dim_cell=256, dim_emb=64):
        super(D_VECTOR, self).__init__()#모듈의 d_vector 상속
        self.lstm = nn.LSTM(input_size=dim_input, hidden_size=dim_cell,
                            num_layers=num_layers, batch_first=True)
        self.embedding = nn.Linear(dim_cell, dim_emb)


    def forward(self, x):
        self.lstm.flatten_parameters()#batch_size, sequence_len, n_feature
        lstm_out, _ = self.lstm(x)#batch_size,sequence_len,hidden_size
        embeds = self.embedding(lstm_out[:,-1,:])#sequence_len의 마지막을 임베딩으로 변환
        norm = embeds.norm(p=2, dim=-1, keepdim=True)
        embeds_normalized = embeds.div(norm)#batch_size, dim_emb
        return embeds_normalized#batch_size, dim_Emb


In [ ]:
#make_metadata.py
"""
Generate speaker embeddings and metadata for training
"""

C = D_VECTOR(dim_input=80, dim_cell=768, dim_emb=256).eval().cuda() #화자별 임베딩
c_checkpoint = torch.load('/content/drive/MyDrive/코딩공부/deep_daiv/dataset/3000000-BL.ckpt')
# c_checkpoint = torch.load('3000000-BL.ckpt')
new_state_dict = OrderedDict()
for key, val in c_checkpoint['model_b'].items():
    new_key = key[7:]
    new_state_dict[new_key] = val
C.load_state_dict(new_state_dict)
num_uttrs = 10
len_crop = 128


# Directory containing mel-spectrograms, 별도 데이터셋?
rootDir = '/content/drive/MyDrive/코딩공부/deep_daiv/dataset/spmel'#?
# rootDir = './spmel'
dirName, subdirList, _ = next(os.walk(rootDir)) #데이터셋 파일 필요함
print('Found directory: %s' % dirName)


speakers = [] #화자 전체의 임베딩 목록
for speaker in sorted(subdirList):
    print('Processing speaker: %s' % speaker)
    utterances = [] #단일 화자별 임베딩을 저장
    utterances.append(speaker)
    _, _, fileList = next(os.walk(os.path.join(dirName,speaker)))#하나씩 추출개시

    # make speaker embedding
    assert len(fileList) >= num_uttrs
    idx_uttrs = np.random.choice(len(fileList), size=num_uttrs, replace=False)
    embs = []
    for i in range(num_uttrs):
        tmp = np.load(os.path.join(dirName, speaker, fileList[idx_uttrs[i]]))
        #디렉토리, 화자, 화자별 파일 순으로 로딩
        candidates = np.delete(np.arange(len(fileList)), idx_uttrs)
        # choose another utterance if the current one is too short
        while tmp.shape[0] < len_crop:#지정 길이보다 짧으면 다른걸로 고름
            idx_alt = np.random.choice(candidates)
            tmp = np.load(os.path.join(dirName, speaker, fileList[idx_alt]))
            candidates = np.delete(candidates, np.argwhere(candidates==idx_alt))
        left = np.random.randint(0, tmp.shape[0]-len_crop)
        melsp = torch.from_numpy(tmp[np.newaxis, left:left+len_crop, :]).cuda()
        #주어진 간격(len_crop)만큼만 잘라서 학습에 활용
        emb = C(melsp)#해당 부분을 임베딩
        embs.append(emb.detach().squeeze().cpu().numpy())#해당 임베딩들 수집
    utterances.append(np.mean(embs, axis=0))#평균값으로 묶어서 화자 정보로 추가

    # create file list
    for fileName in sorted(fileList):#각 화자에 대한 정보 파일로 저장
        utterances.append(os.path.join(speaker,fileName))
    speakers.append(utterances)

with open(os.path.join(rootDir, 'train.pkl'), 'wb') as handle:
    pickle.dump(speakers, handle)

<ipython-input-7-704b09355319>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  c_checkpoint = torch.load('/content/drive/MyDrive/코딩공부/deep_daiv/dataset/3000000-BL.ckp

Found directory: /content/drive/MyDrive/코딩공부/deep_daiv/dataset/spmel
Processing speaker: p225
Processing speaker: p226
Processing speaker: p227
Processing speaker: p228


In [ ]:
#data_loader.py


class Utterances(data.Dataset):
    """Dataset class for the Utterances dataset."""

    def __init__(self, root_dir, len_crop):
        """Initialize and preprocess the Utterances dataset."""
        self.root_dir = root_dir
        self.len_crop = len_crop
        self.step = 10

        metaname = os.path.join(self.root_dir, "train.pkl") #메타데이터 읽어오기
        meta = pickle.load(open(metaname, "rb"))

        """Load data using multiprocessing"""
        manager = Manager()
        meta = manager.list(meta)
        dataset = manager.list(len(meta)*[None])
        processes = []
        for i in range(0, len(meta), self.step):
            p = Process(target=self.load_data,
                        args=(meta[i:i+self.step],dataset,i))#프로세스를 적용할 파일들
            p.start()
            processes.append(p)
        for p in processes:#프로세스 적용 후 저장
            p.join()

        self.train_dataset = list(dataset)
        self.num_tokens = len(self.train_dataset)

        print('Finished loading the dataset...')


    def load_data(self, submeta, dataset, idx_offset):#메타데이터 일부, 데이터셋 리스트, 인덱스 오프셋
        for k, sbmt in enumerate(submeta):
            uttrs = len(sbmt)*[None]#각 화자별로 넣을 수 있는 none칸 생성
            for j, tmp in enumerate(sbmt):#화자별로 ID 임베딩 저장
                if j < 2:  # fill in speaker id and embedding
                    uttrs[j] = tmp
                else: # load the mel-spectrograms

                    #해당 화자의 멜스펙트로그램 읽어옴
                    uttrs[j] = np.load(os.path.join(self.root_dir, tmp),allow_pickle=True)
                    #uttrs[j] = np.load(os.path.join(self.root_dir, tmp))


            dataset[idx_offset+k] = uttrs


    def __getitem__(self, index):#데이터셋의 인덱스
        # pick a random speaker
        dataset = self.train_dataset
        list_uttrs = dataset[index] #데이터셋 상에서 해당 인덱스를 가지는 화자 검색
        emb_org = list_uttrs[1]

        # pick random uttr with random crop
        a = np.random.randint(2, len(list_uttrs)) #무작위 화자 선택
        tmp = list_uttrs[a]
        if tmp.shape[0] < self.len_crop: #추출 길이보다 샘플이 짧음
            len_pad = self.len_crop - tmp.shape[0]#필요 길이까지 0으로 패딩 추가
            uttr = np.pad(tmp, ((0,len_pad),(0,0)), 'constant')
        elif tmp.shape[0] > self.len_crop:
            left = np.random.randint(tmp.shape[0]-self.len_crop)#랜덤위치 잡고 LEN_CROP길이 가져옴
            uttr = tmp[left:left+self.len_crop, :]
        else:
            uttr = tmp#같으면 전체 반환

        return uttr, emb_org


    def __len__(self):
        """Return the number of spkrs."""
        return self.num_tokens #샘플의 토큰 갯수 반환




def get_loader(root_dir, batch_size=16, len_crop=128, num_workers=0):
    """Build and return a data loader."""

    dataset = Utterances(root_dir, len_crop)

    worker_init_fn = lambda x: np.random.seed((torch.initial_seed()) % (2**32))
    data_loader = data.DataLoader(dataset=dataset,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=num_workers,
                                  drop_last=True,
                                  worker_init_fn=worker_init_fn)
    return data_loader

# 모델

In [ ]:
#model_vc.py

class LinearNorm(torch.nn.Module):#가중치 설정용 선형 놈
    def __init__(self, in_dim, out_dim, bias=True, w_init_gain='linear'):
        super(LinearNorm, self).__init__()
        self.linear_layer = torch.nn.Linear(in_dim, out_dim, bias=bias)

        torch.nn.init.xavier_uniform_(#가중치 분산이 입력데이터 갯수에 반비례하게 초기화됨
            self.linear_layer.weight,
            gain=torch.nn.init.calculate_gain(w_init_gain))

    def forward(self, x):
        return self.linear_layer(x)


class ConvNorm(torch.nn.Module): #합성곱 놈
    def __init__(self, in_channels, out_channels, kernel_size=1, stride=1,
                 padding=None, dilation=1, bias=True, w_init_gain='linear'):
        super(ConvNorm, self).__init__()
        if padding is None:
            assert(kernel_size % 2 == 1)
            padding = int(dilation * (kernel_size - 1) / 2)#커널 사이즈에 따라 패딩 조정, 크기 유지

        self.conv = torch.nn.Conv1d(in_channels, out_channels, #1D 컨볼루션 층, 시간따라 바뀌는것만 보면 됨
                                    kernel_size=kernel_size, stride=stride,
                                    padding=padding, dilation=dilation,
                                    bias=bias)

        torch.nn.init.xavier_uniform_(#초기화
            self.conv.weight, gain=torch.nn.init.calculate_gain(w_init_gain))

    def forward(self, signal):
        conv_signal = self.conv(signal)
        return conv_signal


class Encoder(nn.Module): #인코더 모델
    """Encoder module:
    """
    def __init__(self, dim_neck, dim_emb, freq):#보틀넥 구조 디멘션 갯수
        super(Encoder, self).__init__()
        self.dim_neck = dim_neck
        self.freq = freq

        convolutions = []
        for i in range(3):
            conv_layer = nn.Sequential(#컨볼루션 3개 넣음
                ConvNorm(80+dim_emb if i==0 else 512,#입력이 주파수 대역 갯수 + 화자임베딩
                        #또는 원 크기?
                         512,
                         kernel_size=5, stride=1,
                         padding=2,
                         dilation=1, w_init_gain='relu'),
                nn.BatchNorm1d(512))
            convolutions.append(conv_layer)
        self.convolutions = nn.ModuleList(convolutions)

        self.lstm = nn.LSTM(512, dim_neck, 2, batch_first=True, bidirectional=True)
        #쌍방 LSTM

    def forward(self, x, c_org):#배치 사이즈, 1, 시간 프레임 T, 주파수대역 갯수 80
        #C_ORG = 화자 임베딩 1차원 스칼라
        x = x.squeeze(1).transpose(2,1)#배치사이즈, 주파수 대역, 시간 프래임
        c_org = c_org.unsqueeze(-1).expand(-1, -1, x.size(-1))#화자,화자, 시간프레임
        x = torch.cat((x, c_org), dim=1)#

        for conv in self.convolutions:
            x = F.relu(conv(x))
        x = x.transpose(1, 2)

        self.lstm.flatten_parameters()
        outputs, _ = self.lstm(x)
        out_forward = outputs[:, :, :self.dim_neck]
        out_backward = outputs[:, :, self.dim_neck:]

        codes = []
        for i in range(0, outputs.size(1), self.freq):
            codes.append(torch.cat((out_forward[:,i+self.freq-1,:],out_backward[:,i,:]), dim=-1))

        return codes


class Decoder(nn.Module): #디코더 모델
    """Decoder module:
    """
    def __init__(self, dim_neck, dim_emb, dim_pre):
        super(Decoder, self).__init__()

        self.lstm1 = nn.LSTM(dim_neck*2+dim_emb, dim_pre, 1, batch_first=True)

        convolutions = []
        for i in range(3):
            conv_layer = nn.Sequential(
                ConvNorm(dim_pre,
                         dim_pre,
                         kernel_size=5, stride=1,
                         padding=2,
                         dilation=1, w_init_gain='relu'),
                nn.BatchNorm1d(dim_pre))
            convolutions.append(conv_layer)
        self.convolutions = nn.ModuleList(convolutions)

        self.lstm2 = nn.LSTM(dim_pre, 1024, 2, batch_first=True)

        self.linear_projection = LinearNorm(1024, 80)

    def forward(self, x):

        #self.lstm1.flatten_parameters()
        x, _ = self.lstm1(x)
        x = x.transpose(1, 2)

        for conv in self.convolutions:
            x = F.relu(conv(x))
        x = x.transpose(1, 2)

        outputs, _ = self.lstm2(x)

        decoder_output = self.linear_projection(outputs)

        return decoder_output


class Postnet(nn.Module): #포스트넷?
    """Postnet
        - Five 1-d convolution with 512 channels and kernel size 5
    """

    def __init__(self):
        super(Postnet, self).__init__()
        self.convolutions = nn.ModuleList()

        self.convolutions.append(
            nn.Sequential(
                ConvNorm(80, 512,
                         kernel_size=5, stride=1,
                         padding=2,
                         dilation=1, w_init_gain='tanh'),
                nn.BatchNorm1d(512))
        )

        for i in range(1, 5 - 1):
            self.convolutions.append(
                nn.Sequential(
                    ConvNorm(512,
                             512,
                             kernel_size=5, stride=1,
                             padding=2,
                             dilation=1, w_init_gain='tanh'),
                    nn.BatchNorm1d(512))
            )

        self.convolutions.append(
            nn.Sequential(
                ConvNorm(512, 80,
                         kernel_size=5, stride=1,
                         padding=2,
                         dilation=1, w_init_gain='linear'),
                nn.BatchNorm1d(80))
            )

    def forward(self, x):
        for i in range(len(self.convolutions) - 1):
            x = torch.tanh(self.convolutions[i](x))

        x = self.convolutions[-1](x)

        return x


class Generator(nn.Module): #생성기 = 인코더+디코더+포스트넷
    """Generator network."""
    def __init__(self, dim_neck, dim_emb, dim_pre, freq):
        super(Generator, self).__init__()

        self.encoder = Encoder(dim_neck, dim_emb, freq)
        self.decoder = Decoder(dim_neck, dim_emb, dim_pre)
        self.postnet = Postnet()

    def forward(self, x, c_org, c_trg):

        codes = self.encoder(x, c_org)
        if c_trg is None:
            return torch.cat(codes, dim=-1)

        tmp = []
        for code in codes:
            tmp.append(code.unsqueeze(1).expand(-1,int(x.size(1)/len(codes)),-1))
        code_exp = torch.cat(tmp, dim=1)

        encoder_outputs = torch.cat((code_exp, c_trg.unsqueeze(1).expand(-1,x.size(1),-1)), dim=-1)

        mel_outputs = self.decoder(encoder_outputs)

        mel_outputs_postnet = self.postnet(mel_outputs.transpose(2,1))
        mel_outputs_postnet = mel_outputs + mel_outputs_postnet.transpose(2,1)

        mel_outputs = mel_outputs.unsqueeze(1)
        mel_outputs_postnet = mel_outputs_postnet.unsqueeze(1)

        return mel_outputs, mel_outputs_postnet, torch.cat(codes, dim=-1)



# 학습

In [ ]:
#main.py

def str2bool(v):
    return v.lower() in ('true')


In [ ]:
cd /content/drive/MyDrive/코딩공부/deep_daiv/dataset

/content/drive/MyDrive/코딩공부/deep_daiv/dataset


In [ ]:
import sys

In [ ]:
sys.argv=['']

parser = argparse.ArgumentParser()

# Model configuration.
parser.add_argument('--lambda_cd', type=float, default=1, help='weight for hidden code loss')
parser.add_argument('--dim_neck', type=int, default=16)
parser.add_argument('--dim_emb', type=int, default=256)
parser.add_argument('--dim_pre', type=int, default=512)
parser.add_argument('--freq', type=int, default=16)

# Training configuration.
parser.add_argument('--data_dir', type=str, default='./spmel')
parser.add_argument('--batch_size', type=int, default=2, help='mini-batch size')
parser.add_argument('--num_iters', type=int, default=1000000, help='number of total iterations')
parser.add_argument('--len_crop', type=int, default=128, help='dataloader output sequence length')

# Miscellaneous.
parser.add_argument('--log_step', type=int, default=10)

config = parser.parse_args()

In [ ]:
# from easydict import EasyDict as edict

# config = edict()

# # Model configuration.
# config.lambda_cd = 1
# config.dim_neck = 16
# config.dim_emb = 256
# config.dim_pre = 512
# config.freq = 16

# # Training configuration.
# config.data_dir = '/content/drive/MyDrive/코딩공부/deep_daiv/dataset/wavs'
# config.batch_size = 2
# config.num_iters = 1000000
# config.len_crop = 128

# # Miscellaneous.
# config.log_step = 10

In [ ]:
#solver_encoder.py

class Solver(object): #음성신호 학습

    def __init__(self, vcc_loader, config):
        """Initialize configurations."""

        # Data loader.
        self.vcc_loader = vcc_loader

        # Model configurations.
        self.lambda_cd = config.lambda_cd
        self.dim_neck = config.dim_neck
        self.dim_emb = config.dim_emb
        self.dim_pre = config.dim_pre
        self.freq = config.freq

        # Training configurations.
        self.batch_size = config.batch_size
        self.num_iters = config.num_iters

        # Miscellaneous.
        self.use_cuda = torch.cuda.is_available()
        self.device = torch.device('cuda:0' if self.use_cuda else 'cpu')
        self.log_step = config.log_step

        # Build the model and tensorboard.
        self.build_model()


    def build_model(self):

        self.G = Generator(self.dim_neck, self.dim_emb, self.dim_pre, self.freq)

        self.g_optimizer = torch.optim.Adam(self.G.parameters(), 0.0001)

        self.G.to(self.device)


    def reset_grad(self):
        """Reset the gradient buffers."""
        self.g_optimizer.zero_grad()


    #=====================================================================================================================================#



    def train(self):
        # Set data loader.
        data_loader = self.vcc_loader

        # Print logs in specified order
        keys = ['G/loss_id','G/loss_id_psnt','G/loss_cd']

        # Start training.
        print('Start training...')
        start_time = time.time()
        for i in range(self.num_iters):

            # =================================================================================== #
            #                             1. Preprocess input data                                #
            # =================================================================================== #

            # Fetch data.
            try:
                x_real, emb_org = next(data_iter)
            except:
                data_iter = iter(data_loader)
                x_real, emb_org = next(data_iter)



            x_real = x_real.to(self.device)
            emb_org = emb_org.to(self.device)


            # =================================================================================== #
            #                               2. Train the generator                                #
            # =================================================================================== #

            self.G = self.G.train()

            # Identity mapping loss
            x_identic, x_identic_psnt, code_real = self.G(x_real, emb_org, emb_org)
            g_loss_id = F.mse_loss(x_real, x_identic)
            g_loss_id_psnt = F.mse_loss(x_real, x_identic_psnt)

            # Code semantic loss.
            code_reconst = self.G(x_identic_psnt, emb_org, None)
            g_loss_cd = F.l1_loss(code_real, code_reconst)


            # Backward and optimize.
            g_loss = g_loss_id + g_loss_id_psnt + self.lambda_cd * g_loss_cd
            self.reset_grad()
            g_loss.backward()
            self.g_optimizer.step()

            # Logging.
            loss = {}
            loss['G/loss_id'] = g_loss_id.item()
            loss['G/loss_id_psnt'] = g_loss_id_psnt.item()
            loss['G/loss_cd'] = g_loss_cd.item()

            # =================================================================================== #
            #                                 4. Miscellaneous                                    #
            # =================================================================================== #

            # Print out training information.
            if (i+1) % self.log_step == 0:
                et = time.time() - start_time
                et = str(datetime.timedelta(seconds=et))[:-7]
                log = "Elapsed [{}], Iteration [{}/{}]".format(et, i+1, self.num_iters)
                for tag in keys:
                    log += ", {}: {:.4f}".format(tag, loss[tag])
                print(log)







In [ ]:
cudnn.benchmark = True

    # Data loader.
vcc_loader = get_loader(config.data_dir, config.batch_size, config.len_crop)

solver = Solver(vcc_loader, config)

solver.train()



Finished loading the dataset...
Start training...


<ipython-input-19-0a75626680c8>:86: UserWarning: Using a target size (torch.Size([2, 1, 128, 80])) that is different to the input size (torch.Size([2, 128, 80])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  g_loss_id = F.mse_loss(x_real, x_identic)
<ipython-input-19-0a75626680c8>:87: UserWarning: Using a target size (torch.Size([2, 1, 128, 80])) that is different to the input size (torch.Size([2, 128, 80])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  g_loss_id_psnt = F.mse_loss(x_real, x_identic_psnt)


Elapsed [0:00:02], Iteration [10/1000000], G/loss_id: 0.1532, G/loss_id_psnt: 1.1175, G/loss_cd: 0.1230
Elapsed [0:00:03], Iteration [20/1000000], G/loss_id: 0.0594, G/loss_id_psnt: 0.9366, G/loss_cd: 0.1170
Elapsed [0:00:04], Iteration [30/1000000], G/loss_id: 0.1137, G/loss_id_psnt: 0.6641, G/loss_cd: 0.1126
Elapsed [0:00:04], Iteration [40/1000000], G/loss_id: 0.1895, G/loss_id_psnt: 0.4757, G/loss_cd: 0.1096
Elapsed [0:00:05], Iteration [50/1000000], G/loss_id: 0.3117, G/loss_id_psnt: 0.2996, G/loss_cd: 0.0920
Elapsed [0:00:06], Iteration [60/1000000], G/loss_id: 0.2179, G/loss_id_psnt: 0.3585, G/loss_cd: 0.0848
Elapsed [0:00:07], Iteration [70/1000000], G/loss_id: 0.2552, G/loss_id_psnt: 0.3200, G/loss_cd: 0.0782
Elapsed [0:00:07], Iteration [80/1000000], G/loss_id: 0.2784, G/loss_id_psnt: 0.2824, G/loss_cd: 0.0626
Elapsed [0:00:08], Iteration [90/1000000], G/loss_id: 0.2482, G/loss_id_psnt: 0.3102, G/loss_cd: 0.0723
Elapsed [0:00:09], Iteration [100/1000000], G/loss_id: 0.2492, G

KeyboardInterrupt: 

# 보코더


In [ ]:
import torch
import soundfile as sf
import pickle
from tqdm import tqdm
from wavenet_vocoder import builder

In [ ]:
#hparams.py
# NOTE: If you want full control for model architecture. please take a look
# at the code and change whatever you want. Some hyper parameters are hardcoded.


class Map(dict):
	"""
    Example:
    m = Map({'first_name': 'Eduardo'}, last_name='Pool', age=24, sports=['Soccer'])

    Credits to epool:
    https://stackoverflow.com/questions/2352181/how-to-use-a-dot-to-access-members-of-dictionary
    """

	def __init__(self, *args, **kwargs):
		super(Map, self).__init__(*args, **kwargs)
		for arg in args:
			if isinstance(arg, dict):
				for k, v in arg.items():
					self[k] = v

		if kwargs:
			for k, v in kwargs.iteritems():
				self[k] = v

	def __getattr__(self, attr):
		return self.get(attr)

	def __setattr__(self, key, value):
		self.__setitem__(key, value)

	def __setitem__(self, key, value):
		super(Map, self).__setitem__(key, value)
		self.__dict__.update({key: value})

	def __delattr__(self, item):
		self.__delitem__(item)

	def __delitem__(self, key):
		super(Map, self).__delitem__(key)
		del self.__dict__[key]


# Default hyperparameters:
hparams = Map({ #하이퍼 파라미터 묶음
	'name': "wavenet_vocoder",

	# Convenient model builder
	'builder': "wavenet",

	# Input type:
	# 1. raw [-1, 1]
	# 2. mulaw [-1, 1]
	# 3. mulaw-quantize [0, mu]
	# If input_type is raw or mulaw, network assumes scalar input and
	# discretized mixture of logistic distributions output, otherwise one-hot
	# input and softmax output are assumed.
	# **NOTE**: if you change the one of the two parameters below, you need to
	# re-run preprocessing before training.
	'input_type': "raw",
	'quantize_channels': 65536,  # 65536 or 256

	# Audio:
	'sample_rate': 16000,
	# this is only valid for mulaw is True
	'silence_threshold': 2,
	'num_mels': 80,
	'fmin': 125,
	'fmax': 7600,
	'fft_size': 1024,
	# shift can be specified by either hop_size or frame_shift_ms
	'hop_size': 256,
	'frame_shift_ms': None,
	'min_level_db': -100,
	'ref_level_db': 20,
	# whether to rescale waveform or not.
	# Let x is an input waveform, rescaled waveform y is given by:
	# y = x / np.abs(x).max() * rescaling_max
	'rescaling': True,
	'rescaling_max': 0.999,
	# mel-spectrogram is normalized to [0, 1] for each utterance and clipping may
	# happen depends on min_level_db and ref_level_db, causing clipping noise.
	# If False, assertion is added to ensure no clipping happens.o0
	'allow_clipping_in_normalization': True,

	# Mixture of logistic distributions:
	'log_scale_min': float(-32.23619130191664),

	# Model:
	# This should equal to `quantize_channels` if mu-law quantize enabled
	# otherwise num_mixture * 3 (pi, mean, log_scale)
	'out_channels': 10 * 3,
	'layers': 24,
	'stacks': 4,
	'residual_channels': 512,
	'gate_channels': 512,  # split into 2 gropus internally for gated activation
	'skip_out_channels': 256,
	'dropout': 1 - 0.95,
	'kernel_size': 3,
	# If True, apply weight normalization as same as DeepVoice3
	'weight_normalization': True,
	# Use legacy code or not. Default is True since we already provided a model
	# based on the legacy code that can generate high-quality audio.
	# Ref: https://github.com/r9y9/wavenet_vocoder/pull/73
	'legacy': True,

	# Local conditioning (set negative value to disable))
	'cin_channels': 80,
	# If True, use transposed convolutions to upsample conditional features,
	# otherwise repeat features to adjust time resolution
	'upsample_conditional_features': True,
	# should np.prod(upsample_scales) == hop_size
	'upsample_scales': [4, 4, 4, 4],
	# Freq axis kernel size for upsampling network
	'freq_axis_kernel_size': 3,

	# Global conditioning (set negative value to disable)
	# currently limited for speaker embedding
	# this should only be enabled for multi-speaker dataset
	'gin_channels': -1,  # i.e., speaker embedding dim
	'n_speakers': -1,

	# Data loader
	'pin_memory': True,
	'num_workers': 2,

	# train/test
	# test size can be specified as portion or num samples
	'test_size': 0.0441,  # 50 for CMU ARCTIC single speaker
	'test_num_samples': None,
	'random_state': 1234,

	# Loss

	# Training:
	'batch_size': 2,
	'adam_beta1': 0.9,
	'adam_beta2': 0.999,
	'adam_eps': 1e-8,
	'amsgrad': False,
	'initial_learning_rate': 1e-3,
	# see lrschedule.py for available lr_schedule
	'lr_schedule': "noam_learning_rate_decay",
	'lr_schedule_kwargs': {},  # {"anneal_rate": 0.5, "anneal_interval": 50000},
	'nepochs': 2000,
	'weight_decay': 0.0,
	'clip_thresh': -1,
	# max time steps can either be specified as sec or steps
	# if both are None, then full audio samples are used in a batch
	'max_time_sec': None,
	'max_time_steps': 8000,
	# Hold moving averaged parameters and use them for evaluation
	'exponential_moving_average': True,
	# averaged = decay * averaged + (1 - decay) * x
	'ema_decay': 0.9999,

	# Save
	# per-step intervals
	'checkpoint_interval': 10000,
	'train_eval_interval': 10000,
	# per-epoch interval
	'test_eval_epoch_interval': 5,
	'save_optimizer_state': True,

	# Eval:
})


# def hparams_debug_string():
# 	values = hparams.values()
# 	hp = ['  %s: %s' % (name, values[name]) for name in sorted(values)]
# 	return 'Hyperparameters:\n' + '\n'.join(hp)

In [ ]:
torch.set_num_threads(4)
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
def build_model(): #모델 생성

    model = getattr(builder, hparams.builder)(
        out_channels=hparams.out_channels,
        layers=hparams.layers,
        stacks=hparams.stacks,
        residual_channels=hparams.residual_channels,
        gate_channels=hparams.gate_channels,
        skip_out_channels=hparams.skip_out_channels,
        cin_channels=hparams.cin_channels,
        gin_channels=hparams.gin_channels,
        weight_normalization=hparams.weight_normalization,
        n_speakers=hparams.n_speakers,
        dropout=hparams.dropout,
        kernel_size=hparams.kernel_size,
        upsample_conditional_features=hparams.upsample_conditional_features,
        upsample_scales=hparams.upsample_scales,
        freq_axis_kernel_size=hparams.freq_axis_kernel_size,
        scalar_input=True,
        legacy=hparams.legacy,
    )
    return model



def wavegen(model, c=None, tqdm=tqdm): #파형 생성
    """Generate waveform samples by WaveNet.

    """

    model.eval()
    model.make_generation_fast_()

    Tc = c.shape[0]
    upsample_factor = hparams.hop_size
    # Overwrite length according to feature size
    length = Tc * upsample_factor

    # B x C x T
    c = torch.FloatTensor(c.T).unsqueeze(0)

    initial_input = torch.zeros(1, 1, 1).fill_(0.0)

    # Transform data to GPU
    initial_input = initial_input.to(device)
    c = None if c is None else c.to(device)

    with torch.no_grad():
        y_hat = model.incremental_forward(
            initial_input, c=c, g=None, T=length, tqdm=tqdm, softmax=True, quantize=True,
            log_scale_min=hparams.log_scale_min)

    y_hat = y_hat.view(-1).cpu().data.numpy()

    return y_hat

In [ ]:
#synthesis.py
# coding: utf-8
"""
Synthesis waveform from trained WaveNet.

Modified from https://github.com/r9y9/wavenet_vocoder
"""

torch.set_num_threads(4)
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


In [ ]:
spect_vc = pickle.load(open('results.pkl', 'rb'))
device = torch.device("cuda")
model = build_model().to(device)
checkpoint = torch.load("checkpoint_step001000000_ema.pth")
model.load_state_dict(checkpoint["state_dict"])

for spect in spect_vc:
    name = spect[0]
    c = spect[1]
    print(name)
    waveform = wavegen(model, c=c)
    sf.write(name+'.wav', waveform, samplerate=16000)

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
<ipython-input-26-a6ad4607df90>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case wh

p225xp225


100%|██████████| 23040/23040 [04:54<00:00, 78.23it/s]


p225xp228


100%|██████████| 23040/23040 [04:50<00:00, 79.38it/s]


p225xp256


100%|██████████| 23040/23040 [04:50<00:00, 79.34it/s]


p225xp270


 61%|██████    | 13942/23040 [02:53<01:53, 80.21it/s]


KeyboardInterrupt: 